In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pingouin as pg
import seaborn as sns
import os

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_excel('/home/ds/DCL/Alexia/196_opto_individualcells.xlsx')

data_col = df.columns[0]
cell_id_col = df.columns[1]
sweep_id_col = df.columns[2]
stim_id_col = df.columns[3]
voltage_id_col = df.columns[4]
measurement_id_col = df.columns[5]

In [ ]:
def get_stars(p):
    if p <= 0.001:
        s = '***'
    elif p <= 0.01:
        s = '**'
    elif p <= 0.05:
        s = '*'
    else: 
        s = 'n.s.'
    return s
    

In [ ]:
def plot_with_stats(df_tmp, ax):
    df_stim_a = df_tmp.loc[df_tmp[stim_id_col] == stim_a].copy()
    stats_stim_a = pg.wilcoxon(df_stim_a[data_col].values, correction = 'auto')
    pval_stim_a = stats_stim_a.loc['Wilcoxon', 'p-val']
    stars_stim_a = get_stars(pval_stim_a)
    
    df_stim_b = df_tmp.loc[df_tmp[stim_id_col] == stim_b].copy()
    stats_stim_b = pg.wilcoxon(df_stim_b[data_col].values, correction = 'auto')
    pval_stim_b = stats_stim_b.loc['Wilcoxon', 'p-val']
    stars_stim_b = get_stars(pval_stim_b)
    
    sns.stripplot(data = df_tmp, x = stim_id_col, y = data_col, order = [stim_a, stim_b], s=7)
    
    ylim_lower, ylim_upper = ax.get_ylim()
    y_axis_span = ylim_upper - ylim_lower
    y_stars = df_tmp[data_col].max() + y_axis_span*0.1
    ylim_upper_new = ylim_upper + y_axis_span*0.15
    
    plt.text(0, y_stars, stars_stim_a, ha='center', va='bottom', color='k')
    plt.text(1, y_stars, stars_stim_b, ha='center', va='bottom', color='k')
    
    plt.hlines(0, -0.5, 1.5, colors='grey', linestyle='dashed')
    plt.ylim(ylim_lower, ylim_upper_new)
    
    plt.ylabel(measurement_id)
    
    

In [ ]:
df

In [ ]:
#filename = 'excel_sheet'
#out_dir = 'path_to_dir_where_all_plots_are_saved/'
stim_a = 4
stim_b = 20



for cell_id in df[cell_id_col].unique()[:2]:
    fig = plt.figure(figsize=(10, 5), facecolor='white')
    gs = fig.add_gridspec(1,4)
    gs_col = 0
    for voltage_id in df.loc[df[cell_id_col] == cell_id, voltage_id_col].unique():
        for measurement_id in df.loc[(df[cell_id_col] == cell_id) & (df[voltage_id_col] == voltage_id), measurement_id_col].unique():
            df_temp = df.loc[(df[cell_id_col] == cell_id) 
                             & (df[voltage_id_col] == voltage_id) 
                             & (df[measurement_id_col] == measurement_id)].copy()
            plot_with_stats(df_temp, fig.add_subplot(gs[0, gs_col]))
            gs_col = gs_col + 1
    fig.suptitle(cell_id)
    plt.tight_layout()
    #plt.savefig(out_dir + cell_id + '.png', dpi=300)

In [ ]:
def split_data_in_groups(df, data_col, x_split, x_order, hue_split, hue_order):
    splitted_groups = {}
    for x_group_id in x_order:
        if x_group_id not in splitted_groups.keys():
            splitted_groups[x_group_id] = {}
        for hue_group_id in hue_order:
            if df.loc[(df[x_split] == x_group_id) & (df[hue_split] == hue_group_id), data_col].shape[0] > 0:
                if hue_group_id not in splitted_groups[x_group_id].keys():
                    splitted_groups[x_group_id][hue_group_id] = {'data': np.array([]), 
                                                                 'stats': None,
                                                                 'p-val': None, 
                                                                 'stars': None}
                data = df.loc[(df[x_split] == x_group_id) & (df[hue_split] == hue_group_id), data_col].values
                splitted_groups[x_group_id][hue_group_id]['data'] = np.append(splitted_groups[x_group_id][hue_group_id]['data'], data)
    return splitted_groups

In [ ]:
def one_sample_tests_per_group(splitted_groups, fixed_value):
    for first_lvl_key in splitted_groups.keys():
        for second_lvl_key in splitted_groups[first_lvl_key].keys():
            df_test_results = pg.wilcoxon(splitted_groups[first_lvl_key][second_lvl_key]['data'] - fixed_value, correction = 'auto')
            pval = df_test_results.loc['Wilcoxon', 'p-val']
            if pval <= 0.001:
                stars = '***'
            elif pval <= 0.01:
                stars = '**'
            elif pval <= 0.05:
                stars = '*'
            else: 
                stars = 'n.s.'
            splitted_groups[first_lvl_key][second_lvl_key]['stats'] = df_test_results.copy()
            splitted_groups[first_lvl_key][second_lvl_key]['p-val'] = pval
            splitted_groups[first_lvl_key][second_lvl_key]['stars'] = stars
    return splitted_groups
            

In [ ]:
def annotations(stat_results, hue_order):
    annotations = {}
    group_id = 0
    l_first_lvl_keys = list(stat_results.keys())
    for first_lvl_key in l_first_lvl_keys:
        group_x_coord = l_first_lvl_keys.index(first_lvl_key)
        for second_lvl_key in stat_results[first_lvl_key].keys():
            annotations[group_id] = {'p-val': stat_results[first_lvl_key][second_lvl_key]['p-val'],
                                     'stars': stat_results[first_lvl_key][second_lvl_key]['stars']}
            if len(hue_order) == 2:
                if hue_order.index(second_lvl_key) == 0:
                    annotations[group_id]['x'] = group_x_coord - 0.2
                elif hue_order.index(second_lvl_key) == 1:
                    annotations[group_id]['x'] = group_x_coord + 0.2
            if len(hue_order) == 3:
                if hue_order.index(second_lvl_key) == 0:
                    annotations[group_id]['x'] = group_x_coord - 0.25
                elif hue_order.index(second_lvl_key) == 1:
                    annotations[group_id]['x'] = group_x_coord
                elif hue_order.index(second_lvl_key) == 2:
                    annotations[group_id]['x'] = group_x_coord + 0.25
            if len(hue_order) == 4:
                if hue_order.index(second_lvl_key) == 0:
                    annotations[group_id]['x'] = group_x_coord - 0.3
                elif hue_order.index(second_lvl_key) == 1:
                    annotations[group_id]['x'] = group_x_coord - 0.1
                elif hue_order.index(second_lvl_key) == 2:
                    annotations[group_id]['x'] = group_x_coord + 0.1
                elif hue_order.index(second_lvl_key) == 3:
                    annotations[group_id]['x'] = group_x_coord + 0.3                
            group_id = group_id + 1
    return annotations

In [ ]:
def plot_annotated_data(df, data_col, x_split, x_order, hue_split, hue_order, fixed_value, d_annotations, show):

    fig = plt.figure(figsize=(8, 5), facecolor='white')
    ax = fig.add_subplot()
    
    for axis in ['top', 'right']:
        ax.spines[axis].set_visible(False)
        
    sns.stripplot(data = df, x = x_split, y = data_col, order = x_order, hue = hue_split, hue_order = hue_order, dodge = True, s=7)   
    
    ylim_lower, ylim_upper = ax.get_ylim()
    y_axis_span = ylim_upper - ylim_lower
    y_stars = df[data_col].max() + y_axis_span*0.15
    y_pval = df[data_col].max() + y_axis_span*0.1
    ylim_upper_new = ylim_upper + y_axis_span*0.2

    for key in d_annotations.keys():
        plt.text(d_annotations[key]['x'], y_stars, d_annotations[key]['stars'], ha='center', va='bottom', color='k')
        plt.text(d_annotations[key]['x'], y_pval, '({})'.format(str(round(d_annotations[key]['p-val'], 2))), ha='center', va='bottom', color='k')
        
    xlim_lower, xlim_upper = ax.get_xlim()
    plt.hlines(fixed_value, xlim_lower, xlim_upper, colors='k')
    plt.vlines(0.5, ylim_lower, ylim_upper_new, colors='grey', linestyle='dashed')
    plt.vlines(1.5, ylim_lower, ylim_upper_new, colors='grey', linestyle='dashed')
    
    plt.ylim(ylim_lower, ylim_upper_new)
    
    plt.legend(loc='center left', title=hue_split, bbox_to_anchor=(1, 0.5), frameon=False)
    
    if fixed_value == 1:
        norm = 'normalized'
        plt.ylabel('{} HR [mean stim HR / mean baseline HR]'.format(norm))
    else:
        norm = 'delta'
        plt.ylabel('{} HR [mean stim HR - mean baseline HR]'.format(norm))
    
    group_id = df['group_id'].unique()[0]
    plt.title('{} - {}'.format(group_id, norm), pad=20) 
    
    plt.tight_layout()
    filename = os.getcwd() + '/Plots/{}_{}_per_{}_{}.png'.format(group_id, hue_split, x_split, norm)
    plt.savefig(filename, dpi=300)
    if show:
        plt.show()
    else:
        plt.close()
    
    

In [ ]:
l_filenames = ['ResultsECG_Vglut1#910.xlsx', 'ResultsECG_Vglut2#1213.xlsx', 'ResultsECG_Chat#60-63.xlsx','Summary_median HCN4#123132.xlsx']

#l_filenames = ['Summary_median HCN4#123132.xlsx']

show = True

for filename in l_filenames:
    if filename == 'ResultsECG_Vglut2#1213.xlsx':
        stim_order = ['10Hz_10ms_30s', '20Hz_10ms_30s', 'Cst_30s']
    elif filename == 'ResultsECG_Vglut1#910.xlsx':
        stim_order = ['10Hz_10ms_30s', '20Hz_10ms_30s', 'cst_30s']
    elif filename == 'ResultsECG_Chat#60-63.xlsx':
        stim_order = ['10Hz_10ms_30s', '50Hz_15ms_30s', 'cst_30s']
    elif filename == 'Summary_median HCN4#123132.xlsx':
        stim_order = ['15Hz_15ms_30s', '15Hz_30ms_30s', 'Cst_30s']
    
    stim_split = 'stimulus_id'

    session_split = 'session_id'
    session_order = [1, 2, 3]

    position_split = 'position_id'
    position_order = [1, 2, 3]

    subject_split = 'subject_id'
    subject_order = None

    # x_split, x_order, hue_split, hue_order
    l_params = [(stim_split, stim_order, position_split, position_order),
                (stim_split, stim_order, session_split, session_order),

                (position_split, position_order, stim_split, stim_order),
                (position_split, position_order, session_split, session_order),

                # Subject hue_splits:
                (stim_split, stim_order, subject_split, subject_order),
                (position_split, position_order, subject_split, subject_order),
                (session_split, session_order, subject_split, subject_order)]
    
    filedir = '/home/ds/DCL/Cardiac_opto/Compare_experimental_conditions_data/'
    d_dfs = pd.read_excel(filedir + filename, sheet_name=None, index_col=0)

    key_overview, key_norm, key_delta = d_dfs.keys()
    for df_key in [key_norm, key_delta]:
        if df_key == key_norm:
            fixed_value = 1
        else:
            fixed_value = 0

        df = d_dfs[df_key]
        data_col = list(df.columns)[0]

        for params in l_params:
            x_split, x_order, hue_split, hue_order = params

            if hue_split == 'subject_id':
                hue_order = list(df[hue_split].unique())

            splitted_groups = split_data_in_groups(df, data_col, x_split, x_order, hue_split, hue_order)
            stat_results = one_sample_tests_per_group(splitted_groups, fixed_value)
            d_annotations = annotations(stat_results, hue_order)
            plot_annotated_data(df, data_col, x_split, x_order, hue_split, hue_order, fixed_value, d_annotations, show)

# More detailed look at responders

In [ ]:
l_filenames = ['ResultsECG_Vglut1#910.xlsx', 'ResultsECG_Chat#60-63.xlsx', 'ResultsECG_Vglut2#1213.xlsx']

for filename in l_filenames:
    if filename == 'ResultsECG_Vglut1#910.xlsx':
        subject_id = '#9'
    elif filename == 'ResultsECG_Chat#60-63.xlsx':
        subject_id = '#62'
    elif filename == 'ResultsECG_Vglut2#1213.xlsx':
        subject_id = '#13'

    filedir = '/home/ds/DCL/Cardiac_opto/Compare_experimental_conditions_data/'
    d_dfs = pd.read_excel(filedir + filename, sheet_name=None, index_col=0)  
    key_overview, key_norm, key_delta = d_dfs.keys()
    for key in [key_norm, key_delta]:
        df_temp = d_dfs[key].loc[d_dfs[key]['subject_id'] == subject_id]
        data_col = df_temp.columns[0]
        group_id = df_temp.group_id.unique()[0]
        
        fig = plt.figure(figsize=(8, 5), facecolor='white')
        ax = fig.add_subplot()

        for axis in ['top', 'right']:
            ax.spines[axis].set_visible(False)
        
        sns.stripplot(data=df_temp, x='stimulus_id', y=data_col, hue='position_id', dodge=True, s=7)
        if key == key_norm:
            fixed_val = 1
            norm = 'normalized'
            plt.ylabel('{} HR [mean stim HR / mean baseline HR]'.format(norm))
        elif key == key_delta:
            fixed_val = 0
            norm = 'delta'
            plt.ylabel('{} HR [mean stim HR - mean baseline HR]'.format(norm))
        
        x0, x1 = ax.get_xlim()
        y0, y1 = ax.get_ylim()
        plt.ylim(y0, y1)
        
        plt.hlines(fixed_val, x0, x1, color='k')
        plt.vlines(0.5, y0, y1, color='gray', linestyle='dashed')
        plt.vlines(1.5, y0, y1, color='gray', linestyle='dashed')
        plt.legend(loc='center left', title='position_id', bbox_to_anchor=(1, 0.5), frameon=False)
        plt.title('{}: mouse {} - {}'.format(group_id, subject_id, norm))
        plt.tight_layout()
        img_filename = os.getcwd() + '/Plots/Responders/{}_mouse{}_positionID_per_stimulusID_{}.png'.format(group_id, subject_id, norm)
        plt.savefig(img_filename, dpi=300)
        plt.show()
    print('\n \n')

# Varianzanalyse

In [ ]:
l_filenames = ['ResultsECG_Vglut1#910.xlsx', 'ResultsECG_Vglut2#1213.xlsx', 'ResultsECG_Chat#60-63.xlsx','Summary_median HCN4#123132.xlsx']

l_norm_dfs = []
l_delta_dfs = []

for filename in l_filenames:
    filedir = '/home/ds/DCL/Cardiac_opto/Compare_experimental_conditions_data/'
    d_dfs = pd.read_excel(filedir + filename, sheet_name=None, index_col=0)    
    
    key_overview, key_norm, key_delta = d_dfs.keys()
    l_cols = list(d_dfs[key_norm].columns)
    l_cols[0] = 'data'
    d_dfs[key_norm].columns = l_cols
    d_dfs[key_delta].columns = l_cols
    l_norm_dfs.append(d_dfs[key_norm])
    l_delta_dfs.append(d_dfs[key_delta])
    
df_norm = pd.concat(l_norm_dfs, axis=0)
df_delta = pd.concat(l_delta_dfs, axis=0)

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='white')
ax = fig.add_subplot()

for axis in ['top', 'right']:
    ax.spines[axis].set_visible(False)

sns.stripplot(data=df_delta, x='position_id', y='data', s=7)

x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
plt.ylim(y0, y1)
plt.ylabel('delta HR [mean stim HR - mean baseline HR]')
plt.hlines(0, x0, x1, color='k')
plt.vlines(0.5, y0, y1, color='gray', linestyle='dashed')
plt.vlines(1.5, y0, y1, color='gray', linestyle='dashed')
plt.title('all data - delta')
plt.tight_layout()
img_filename = os.getcwd() + '/Plots/Variance/all_data_per_positionID_delta_stripplot.png'
plt.savefig(img_filename, dpi=300)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='white')
ax = fig.add_subplot()

for axis in ['top', 'right']:
    ax.spines[axis].set_visible(False)

sns.stripplot(data=df_delta, x='position_id', y='data', color='k', alpha=0.5, s=7)
sns.violinplot(data=df_delta, x='position_id', y='data')

x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
plt.ylim(y0, y1)
plt.ylabel('delta HR [mean stim HR - mean baseline HR]')
plt.hlines(0, x0, x1, color='k')
plt.vlines(0.5, y0, y1, color='gray', linestyle='dashed')
plt.vlines(1.5, y0, y1, color='gray', linestyle='dashed')
plt.title('all data - delta')
plt.tight_layout()
img_filename = os.getcwd() + '/Plots/Variance/all_data_per_positionID_delta_violinplot.png'
plt.savefig(img_filename, dpi=300)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='white')
ax = fig.add_subplot()

for axis in ['top', 'right']:
    ax.spines[axis].set_visible(False)

sns.stripplot(data=df_delta, x='position_id', y='data', hue='group_id', dodge=True, s=7)

x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
plt.ylim(y0, y1)
plt.ylabel('delta HR [mean stim HR - mean baseline HR]')
plt.hlines(0, x0, x1, color='k')
plt.vlines(0.5, y0, y1, color='gray', linestyle='dashed')
plt.vlines(1.5, y0, y1, color='gray', linestyle='dashed')
plt.legend(loc='center left', title='group_id', bbox_to_anchor=(1, 0.5), frameon=False)
plt.title('all data - delta')
plt.tight_layout()
img_filename = os.getcwd() + '/Plots/Variance/all_data_groupID_per_positionID_delta_stripplot.png'
plt.savefig(img_filename, dpi=300)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6), facecolor='white')
ax = fig.add_subplot()

for axis in ['top', 'right']:
    ax.spines[axis].set_visible(False)

sns.stripplot(data=df_delta, x='position_id', y='data', hue='subject_id', dodge=True, s=7)

x0, x1 = ax.get_xlim()
y0, y1 = ax.get_ylim()
plt.ylim(y0, y1)
plt.ylabel('delta HR [mean stim HR - mean baseline HR]')
plt.hlines(0, x0, x1, color='k')
plt.vlines(0.5, y0, y1, color='gray', linestyle='dashed')
plt.vlines(1.5, y0, y1, color='gray', linestyle='dashed')
plt.legend(loc='center left', title='subject_id', bbox_to_anchor=(1, 0.5), frameon=False)
plt.title('all data - delta')
plt.tight_layout()
img_filename = os.getcwd() + '/Plots/Variance/all_data_subjectID_per_positionID_delta_stripplot.png'
plt.savefig(img_filename, dpi=300)
plt.show()

## Daten von #60 Chat-Cre für Stim 2 fehlen
## Teildaten von #62 Chat-Cre von Session 2 fehlen

In [ ]:
annotations = {}
group_id = 0
l_first_lvl_keys = list(stat_results.keys())
l_second_lvl_keys = []
for first_lvl_key in l_first_lvl_keys:
    l_temp = [elem for elem in stat_results[first_lvl_key].keys() if elem not in l_second_lvl_keys]
    if len(l_temp) > 0:
        for key in l_temp:
            l_second_lvl_keys.append(key)

In [ ]:
l_second_lvl_keys

In [ ]:



splitted_groups = split_data_in_groups(data_norm, 'data (nomalized)', 'stimulus_id', ['10Hz_10ms_30s', '20Hz_10ms_30s', 'Cst_30s'], 'session_id', [1, 2])

In [ ]:
fig = plt.figure(figsize=(10, 7), facecolor='white')
ax = fig.add_subplot()

data_norm = d_dfs['delta'].copy()

sns.stripplot(data=data_norm, x='stimulus_id', y='data (nomalized)', hue='position_id', dodge=True, ax = ax)
y = data_norm['data (nomalized)'].max() * 1.05
y_lim_upper = round(data_norm['data (nomalized)'].max() * 1.1, 0)
y_lim_lower = round(data_norm['data (nomalized)'].min() - data_norm['data (nomalized)'].min() * 0.05, 0)
for key in d_annotations.keys():
    plt.text(d_annotations[key]['x'], y, d_annotations[key]['stars'], ha='center', va='bottom', color='k')
plt.ylim(y_lim_lower,y_lim_upper)
plt.legend(loc='center left', title='position_id', bbox_to_anchor=(1, 0.5), frameon=False)


plt.show()

In [ ]:
splitted_groups

In [ ]:
stat_results = one_sample_tests_per_group(splitted_groups, 0)

In [ ]:
stat_results

In [ ]:
d_annotations = annotations(stat_results)

In [ ]:
d_annotations